- 19/06/24 REDUX version
   Just pulling good bit from previous version!
   
TODO:

- Fit testing and uncertainties, see dev notebook for latter case (not currently giving output...)

- FIX SPELLCHECKER, see https://github.com/jupyterlab-contrib/spellchecker
   - Tested online dict, didn't seem to fix things, but maybe need a restart?
   - Could also just try a rebuild, since this also seemed to be a Jupyter version issue (although may also be that some install stages were missing in current Docker build...?) May also be a built-in one now...?

(page:advanced-fitting)=
# Advanced fitting for hyperfine beat (stage 1 bootstrap)

For advanced fitting, try a stage 1 style bootstrap. In this case, options are:

- "basic" ignore the photoionization dynamics and just try fitting the beat to the $l=4$, ROI=0 case, since it is already pretty close and may be assumed to be directly mapped here. See {ref}`page:basic-fitting`.
- "advanced" set (arbitrary) parameters per final state for the probe, and fit these plus the hyperfine beat model parameters. This should allow for a match to a single set of hyperfine parameters for all observables, and fulfil the stage 1 bootstrap criteria. (This page.)

---

From prior work and data:

- Forbes, R. et al. (2018) ‘Quantum-beat photoelectron-imaging spectroscopy of Xe in the VUV’, Physical Review A, 97(6), p. 063417. Available at: https://doi.org/10.1103/PhysRevA.97.063417. arXiv: http://arxiv.org/abs/1803.01081, Authorea (original HTML version): https://doi.org/10.22541/au.156045380.07795038
- Data (OSF): https://osf.io/ds8mk/
- [Quantum Metrology with Photoelectrons (Github repo)](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons), particularly the [Alignment 3 notebook](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons/blob/master/Alignment/Alignment-3.ipynb). Functions from this notebook have been incorporated in the current project, under `qbanalysis.hyperfine`.

## Advanced model & fitting methodology

### Model improvements

In order to more fully describe the data, make the following additions to the basic model:

1. Allow for signal decay with exponential decay and lifetimes per isotope. This can be added to the hyperfine wavepacket ({eq}`TKQ-wavepacket`) as:

$$
\begin{equation}
\langle T(J;t,\tau_I)_{KQ}^{\dagger} \rangle =G(J;t)_{K}\langle T(J)_{KQ}^{\dagger}\rangle \exp^{(-t/\tau_{I})}
\end{equation}
$$ (TKQ-wavepacket-decay)

Where $\tau_I$ is the lifetime for each isotope/nuclear spin state.

2. Add a phenomenological ionization model. In this case, simply allow for a magnitude and offset per channel (i.e. per $l$, $ROI$ in the dataset). In this case, the signal per channel can be described as the product of the wavepacket and ionization model:

$$
S(l,t,ROI) = [A_{l,ROI} * \sum_{I}\langle T(J;t,\tau_I)_{KQ}^{\dagger} \rangle] + O_{l,ROI}
$$  (S-ltI)

In this model the amplitudes $A$ and offsets $O$ can be +ve or -ve, but no other phase terms are included, and they are assumed to be the same for each isotope (but not ionization channel). Note that the underlying wavepacket is effectively global, aside from the decay constants, as indicated above.


### Fitting changes

For handling larger models and fitting, [lmfit](https://lmfit.github.io) can be used. This provides a method for setting and addressing parameters by name (with class and dictionary syntax), and is therefore easier to use and debug as compared the the base Scipy routines (at the cost of a slightly more elaborate setup and configuration). Note, however, that Scipy routines are still used on the backend, but lmfit provides a convenient  wrapper and ancillary functionality.

## Setup fitting model

Follow the modelling notebook ({ref}`page:hyperfine-beat-model`), but wrap functions for fitting, plus updates to use advanced model + lmfit.

New functions are in `qbanalysis.adv_fitting.py`.

### Imports

In [1]:
# Load packages
# Main functions used herein from qbanalysis.hyperfine
from qbanalysis.hyperfine import *
import numpy as np
from epsproc.sphCalc import setBLMs

from pathlib import Path

dataPath = Path('/tmp/xe_analysis')
# dataTypes = ['BLMall', 'BLMerr', 'BLMerrCycle']   # Read these types, should just do dir scan here.

# # Read from HDF5/NetCDF files
# # TO FIX: this should be identical to loadFinalDataset(dataPath), but gives slightly different plots - possibly complex/real/abs confusion?
# dataDict = {}
# for item in dataTypes:
#     dataDict[item] = IO.readXarray(fileName=f'Xe_dataset_{item}.nc', filePath=dataPath.as_posix()).real
#     dataDict[item].name = item

# Read from raw data files
from qbanalysis.dataset import loadFinalDataset
dataDict = loadFinalDataset(dataPath)

# Use Pandas and load Xe local data (ODS)
# These values were detemermined from the experimental data as detailed in ref. [4].
from qbanalysis.dataset import loadXeProps
xeProps = loadXeProps()

2024-06-22 15:54:27.379 | INFO     | qbanalysis.config:<module>:11 - PROJ_ROOT path is: /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


* sparse not found, sparse matrix forms not available. 
* natsort not found, some sorting functions not available. 


* Setting plotter defaults with epsproc.basicPlotters.setPlotters(). Run directly to modify, or change options in local env.


* Set Holoviews with bokeh.
* pyevtk not found, VTK export not available. 
2024-06-22 15:54:33.196 | INFO     | qbanalysis.hyperfine:<module>:28 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-22 15:54:33.275 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_cycleSummed_rot90_quad1_ROI_results_with_FT_NFFT1024_hanningWindow_270717.mat.
2024-06-22 15:54:33.321 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_allCycles_ROIs_with_FTs_NFFT1024_hanningWindow_270717.mat.
2024-06-22 15:54:33.632 | INFO     | qbanalysis.dataset:loadFinalDataset:220 - Processed data to Xarray OK.
2024-06-22 15:54:33.671 | INFO     | qbanalysis.dataset:loadXeProps:71 - Loaded Xe data from /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/dataLocal/Xe_data_table_fixedFractions.ods.

**Xe measured level splittings and the hyperfine constants.**
Statistical uncer

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

In [2]:
# v2 pkg
from qbanalysis.adv_fitting import * 

2024-06-22 15:54:33.721 | INFO     | qbanalysis.basic_fitting:<module>:21 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-22 15:54:33.722 | INFO     | qbanalysis.adv_fitting:<module>:27 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.


In [3]:
# Hide future warnings from Xarray concat for fitting on some platforms
import warnings
# warnings.filterwarnings('ignore')  # ALL WARNINGS
# warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

### Init parameters

Here the defaults can be used initially, and later modified as desired.

Here: 

- the basic model parameters for the level splittings are `s0, s1...`, 
- the lifetimes per isotope are `tau129` and `tau131`
- $A_{l,I}$, $O_{l,I}$ are denoted `l<N>_<param>_<ROI>`, for $N=2,4$, `<param>=amp,off` and `<ROI>=0,1` to match the channels in the experimental data.

In [4]:
# Create model parameters
params = initParams(xeProps)

# Display default repr
params

name,value,initial value,min,max,vary
s0,0.28630000,0.2863,0.00000000,1.00000000,True
s1,0.08550000,0.0855,0.00000000,1.00000000,True
s2,0.14110000,0.1411,0.00000000,1.00000000,True
s3,0.22760000,0.2276,0.00000000,1.00000000,True
tau129,5000.00000,5000,0.00000000,200000.000,True
tau131,5000.00000,5000,0.00000000,200000.000,True
l2_amp_0,-5.78177399,-5.781773989033474,-10.0000000,10.0000000,True
l4_amp_0,-1.83531330,-1.8353132988776526,-10.0000000,10.0000000,True
l2_offset_0,4.59796728,4.597967279969904,-10.0000000,10.0000000,True
l4_offset_0,-9.74218501,-9.742185009605244,-10.0000000,10.0000000,True


In [5]:
# Parameter values can be modified with params.set()
params.set(l2_amp_0=1)

# Or use dict syntax to chnage other properties
params.set(s1={'max':0.5})
params.set(s2={'vary':False})  # Skip this parameter as it is not actually used


# Properties can be checked by name or .get
print(params['l2_amp_0'])
print(params.get('s1'))

<Parameter 'l2_amp_0', value=1, bounds=[-10:10]>
<Parameter 's1', value=array(0.0855), bounds=[0:0.5]>


## Set data and test model

In [6]:
# Set data for l=2,4, and uncertainties
dataIn = dataDict['BLMall'].unstack().sel({'l':[2,4]}).copy()
dataUn = dataDict['BLMerr'].unstack().sel({'l':[2,4]}).copy()  # Main data has uncertainties separately currently

In [7]:
# Compute advanced model...
calcDict = calcAdvFitModel(params, xePropsFit=xeProps, dataDict=dataDict)

# Note this returns various stages of the calculation in the output dict
calcDict.keys()

dict_keys(['xData', 'xePropsFit', 'dataDict', 'trange', 'fitFlag', 'returnType', 'modelDict', 'modelDictSum', 'modelDA', 'modelSum', 'dataIn', 'modelIn', 'res', 'iso', 'dataCol', 'decay', 'ionization'])

In [8]:
# Plot original (basic) model, and model with decay
plotHyperfineModel(calcDict['modelDA']) * plotHyperfineModel(calcDict['decay'])

:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II  :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (sum,sum_std)
   .NdOverlay.III :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (decay,decay_std)
   .NdOverlay.IV  :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (decay,decay_std)

In [9]:
# Plot ionization model - this now has channels to match experimental case (l,ROI)
plotHyperfineModel(calcDict['ionization'].squeeze(),overlay=['l'])

:HoloMap   [ROI]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (Ionization test,Ionization test_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (Ionization test,Ionization test_std)

## Fitting with the advanced model

Here use `qbanalysis.adv_fitting.calcAdvlmfit()` - this wraps the above model + residual calculations for lmfit.

In [10]:
mini = lmfit.Minimizer(calcAdvlmfit, params, fcn_kws={'xePropsFit':xeProps, 'dataDict':dataDict})  
result = mini.minimize()

In [24]:
# Check final output and plot vs. data
# from qbanalysis.plots import plotFinalDatasetBLMt
# plotOpts = {'width':800}
# calcDict = calcAdvFitModel(out.params, xePropsFit=xeProps, dataDict=dataDict)
# plotHyperfineModel(calcDict['ionization'],overlay=['ROI']) * plotFinalDatasetBLMt(**dataDict, **plotOpts)


# PLOT TESTING...
# TODO: add labels and fix ledgend in layout
from qbanalysis.plots import plotFinalDatasetBLMt
plotOpts = {'width':800}
calcDict = calcAdvFitModel(result.params, xePropsFit=xeProps, dataDict=dataDict)
# plotHyperfineModel(calcDict['ionization'],overlay=['ROI']).layout('l')

# To fix layout issues, treat l separately...
l2 = (plotFinalDatasetBLMt(**dataDict) * plotHyperfineModel(calcDict['ionization'],overlay=['ROI'])).select(l=2)
l4 = (plotFinalDatasetBLMt(**dataDict) * plotHyperfineModel(calcDict['ionization'],overlay=['ROI'])).select(l=4)

(l2.overlay('l').opts(title="l2", **plotOpts) + l4.overlay('l').opts(title="l4", **plotOpts)).cols(1)


:Layout
   .HoloMap.I  :HoloMap   [Q]
      :NdOverlay   [l]
         :Overlay
            .NdOverlay.I   :NdOverlay   [ROI]
               :Spread   [t]   (BLM per cycle,BLM per cycle_std)
            .NdOverlay.II  :NdOverlay   [ROI]
               :Curve   [t]   (BLM input 0)
            .NdOverlay.III :NdOverlay   [ROI]
               :Spread   [t]   (Ionization test,Ionization test_std)
            .NdOverlay.IV  :NdOverlay   [ROI]
               :Curve   [t]   (Ionization test,Ionization test_std)
   .HoloMap.II :HoloMap   [Q]
      :NdOverlay   [l]
         :Overlay
            .NdOverlay.I   :NdOverlay   [ROI]
               :Spread   [t]   (BLM per cycle,BLM per cycle_std)
            .NdOverlay.II  :NdOverlay   [ROI]
               :Curve   [t]   (BLM input 0)
            .NdOverlay.III :NdOverlay   [ROI]
               :Spread   [t]   (Ionization test,Ionization test_std)
            .NdOverlay.IV  :NdOverlay   [ROI]
               :Curve   [t]   (Ionization test,Ionization test_std)

In [22]:
# Check final residuals
res, dataDS, modelDS = residualAdv(calcDict['ionization'].squeeze(), dataIn.squeeze(), dataUn = dataUn)
plotHyperfineModel(res,overlay=['l'], **plotOpts)

:HoloMap   [ROI,m]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (res,res_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (res,res_std)

In [13]:
# Final fit parameters
# result.params.pretty_print()
result.params

name,value,standard error,relative error,initial value,min,max,vary
s0,0.29113545,2.5714e-04,(0.09%),0.2863,0.00000000,1.00000000,True
s1,0.08509319,8.9181e-04,(1.05%),0.0855,0.00000000,0.50000000,True
s2,0.14110000,0.00000000,(0.00%),0.1411,0.00000000,1.00000000,False
s3,0.23093367,7.1832e-04,(0.31%),0.2276,0.00000000,1.00000000,True
tau129,3389.37978,928.502919,(27.39%),5000,0.00000000,200000.000,True
tau131,3765.73312,1321.71669,(35.10%),5000,0.00000000,200000.000,True
l2_amp_0,1.94104440,0.14739855,(7.59%),1,-10.0000000,10.0000000,True
l4_amp_0,1.87624832,0.07115143,(3.79%),-1.8353132988776526,-10.0000000,10.0000000,True
l2_offset_0,0.47571702,0.00759598,(1.60%),4.597967279969904,-10.0000000,10.0000000,True
l4_offset_0,-0.01934221,0.00168126,(8.69%),-9.742185009605244,-10.0000000,10.0000000,True


## Check hyperfine A, B params from the updated model

In [14]:
from qbanalysis.basic_fitting import extractABParams
xePropsFit =extractABParams(calcDict['xePropsFit'])
xePropsFit.style.set_caption("Updated results")

/opt/conda/lib/python3.10/site-packages/scipy/optimize/_lsq/least_squares.py:111: UserWarning: Setting `gtol` below the machine epsilon (2.22e-16) effectively disables the corresponding termination condition.
  warn("Setting `{}` below the machine epsilon ({:.2e}) effectively "


`gtol` termination condition is satisfied.
Function evaluations 16, initial cost 8.6539e-07, final cost 1.2885e-24, first-order optimality 9.83e-19.


In [15]:
xePropsFit.droplevel(['I','F′','F'])[0:2][['A/MHz','B/MHz']]

,A/MHz,B/MHz
Isotope,,
129,-5818.680853,nan+/-nan
131,1740.852483,40.155878


![Xe table](xe_table.png)

## Check uncertainties

In [36]:
from uncertainties import ufloat
ufloat(result.params['s0'].value, result.params['s0'].stderr)


0.29113545207179825+/-0.0002571400366251326

In [37]:
# dir(result.params['s0'])
result.params['s0']

<Parameter 's0', value=0.29113545207179825 +/- 0.000257, bounds=[0:1]>

In [ ]:
# Parse params to Ufloats

## Versions

In [16]:
import scooby
scooby.Report(additional=['qbanalysis','pemtk','epsproc', 'holoviews', 'hvplot', 'xarray', 'matplotlib', 'bokeh'])

--------------------------------------------------------------------------------
  Date: Sat Jun 22 15:55:04 2024 EDT

                OS : Linux
            CPU(s) : 64
           Machine : x86_64
      Architecture : 64bit
               RAM : 62.8 GiB
       Environment : Jupyter
       File system : btrfs

  Python 3.10.11 | packaged by conda-forge | (main, May 10 2023, 18:58:44)
  [GCC 11.3.0]

        qbanalysis : 0.0.1
             pemtk : 0.0.1
           epsproc : 1.3.2-dev
         holoviews : 1.16.2
            hvplot : 0.8.4
            xarray : 2022.3.0
        matplotlib : 3.5.3
             bokeh : 3.1.1
             numpy : 1.23.5
             scipy : 1.10.1
           IPython : 8.13.2
            scooby : 0.7.2
--------------------------------------------------------------------------------

In [17]:
# # Check current Git commit for local ePSproc version
# from pathlib import Path
# !git -C {Path(qbanalysis.__file__).parent} branch
# !git -C {Path(qbanalysis.__file__).parent} log --format="%H" -n 1

In [18]:
# # Check current remote commits
# !git ls-remote --heads https://github.com/phockett/qbanalysis

In [19]:
# Check current Git commit for local code version
import qbanalysis
!git -C {Path(qbanalysis.__file__).parent} branch
!git -C {Path(qbanalysis.__file__).parent} log --format="%H" -n 1

* master
  uncertainties
81b13e16b3c931a65190ea7b362feb27c00b54eb


In [20]:
# Check current remote commits
!git ls-remote --heads https://github.com/phockett/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV

605855b8700ed52015c03676c42ff4fa451a6970	refs/heads/master
2ff23ede221ac1a0ae8b5351c6c505a6ecd1b65d	refs/heads/uncertainties
